In [7]:
# Import libraries

import pandas as pd
from pathlib import Path
from matplotlib import pyplot as plt
import os

In [8]:
# os.chdir('put/directory/here')  ## Only if you want to change to a different directory
xvg_all = sorted(list(Path('../files').glob("rdf*.xvg")))

In [9]:
# ================================
# Plot options
# ================================

font_size = 25
width = 2
params = {
    'axes.labelsize': font_size,
    'axes.linewidth': width,
    'figure.facecolor': 'white',
    'xtick.labelsize': font_size,
    'ytick.labelsize': font_size,
    'font.family' : 'CMU Serif',
    'text.usetex': True, 
    'text.latex.preamble': r'\usepackage{siunitx, mhchem}',
    'lines.linewidth' : 3,
    'lines.linestyle' : '-',
    'lines.marker' : None,
    'lines.markersize' : 10,
    'legend.fontsize': font_size,
    'axes.titlesize': 1.5*font_size,
    'legend.fontsize': font_size,
    'font.family': 'serif',
    'grid.color' : 'gray',
    'figure.figsize' : (12,7)
}

plt.rcParams.update(params)

# ================================
# Figures path
# ================================
fig_path = Path('../figures')
if not fig_path.exists():
    os.mkdir(fig_path)
    
# ================================
# Basic Functions
# ================================

def save_rdfs(xvg_list,label=False,title=False):
    for filename in xvg_list:
        filename = str(filename)
        if not title:
            title = filename.replace('.xvg','').split('_')[-1].capitalize()
            outname = title
        else:
            outname = '_'.join([i.lower() for i in title.split()])
        rdf_dict = {}
        cn_dict = {}
        x = []
        rdf_y = []
        cn_y = []

        with open(filename,'rt') as f:
            file = f.readlines()

        for line in file:
            if not line.startswith('#') and not line.startswith('@'):
                line = line.split()
                x.append(float(line[0].strip()))
                rdf_y.append(float(line[1].strip()))

        with open(filename.replace('rdf','cn'),'rt') as f:
            file = f.readlines()

        for line in file:
            if not line.startswith('#') and not line.startswith('@'):
                line = line.split()
                cn_y.append(float(line[1].strip()))

        rdf_dict['g(r)'] = rdf_y
        rdf_dict['CN'] = cn_y


        rdf_df = pd.DataFrame(rdf_dict)
        rdf_df.index = x
        rdf_df.index.name = 'Distance (\AA)'

        # Getting CN Value

        max_idx = rdf_df['g(r)'].argmax()
        min_idx = rdf_df['g(r)'].iloc[max_idx:].argmin()
        idx = max_idx + min_idx
        cn_value = str(round(rdf_df['CN'].iloc[idx],1))
        if label:
            rdf_df['g(r)'].plot(label='{} ; CN: {}'.format(label,cn_value))
        else:
            rdf_df['g(r)'].plot(label='CN: {}'.format(cn_value))
        plt.legend()
        plt.title(title.replace('_',' '))
        outname = fig_path / outname
        plt.savefig(outname,bbox_inches='tight',dpi=300)

In [10]:
save_rdfs(xvg_all,label='NVT',title='RDF production NVT')
plt.close()